In [ ]:

# --- Setup (hidden) ---
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

# Toggle for small teaching callouts under key visuals
TEACH_MODE = True

# Deterministic synthetic data (teaching project)
rng = np.random.default_rng(42)

GENERATED_AT = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
DATA_THROUGH = date.today().strftime("%Y-%m-%d")

# Target headline numbers (kept consistent with deck narrative)
BASELINE_WEEK_TOTAL = 7_740
CURRENT_WEEK_TOTAL  = 30_831
DELTA_WEEK          = CURRENT_WEEK_TOTAL - BASELINE_WEEK_TOTAL  # 23,091
DELTA_PER_BUS_DAY   = DELTA_WEEK / 5.0                          # 4,618.2
PARTIAL_WEEK_TOTAL  = 5_612
PARTIAL_RATIO       = PARTIAL_WEEK_TOTAL / CURRENT_WEEK_TOTAL   # 18.2%

# Risk bins (single source of truth)
def partial_week_risk_label(ratio: float) -> str:
    if ratio < 0.10:
        return "LOW"
    if ratio < 0.20:
        return "MED"
    return "HIGH"

# Synthetic calendar (12 complete weeks + 1 partial week)
today = date.today()
this_monday = today - timedelta(days=today.weekday())
week_starts = [this_monday - timedelta(weeks=w) for w in range(13, 1, -1)]  # 12 complete weeks ending last week
current_week_start = this_monday - timedelta(weeks=1)  # last complete week
partial_week_start = this_monday  # current incomplete week

segments = ["Payer A","Payer B","Payer C","Payer D","Payer E","Payer F","Payer G"]

baseline_shares = np.array([0.30,0.20,0.15,0.10,0.08,0.09,0.08])  # sums 1.00
current_shares  = np.array([0.375,0.19,0.14,0.09,0.075,0.07,0.06]) # sums 1.00

def make_week(week_start: date, total: int, shares: np.ndarray, complete: bool=True) -> pd.DataFrame:
    # Create daily volumes with a mild DOW pattern
    dows = np.arange(7)
    base = np.array([0.13,0.15,0.16,0.15,0.14,0.14,0.13])  # sums 1.00
    if not complete:
        dows = np.arange(4)  # Mon-Thu present
        base = base[:4]
        base = base / base.sum()
    daily = np.round(total * base).astype(int)
    # Fix rounding to match exact total
    daily[-1] += total - daily.sum()

    rows=[]
    for di, dow in enumerate(dows):
        dt = week_start + timedelta(days=int(dow))
        # segment split per day
        seg_counts = np.round(daily[di] * shares).astype(int)
        seg_counts[-1] += daily[di] - seg_counts.sum()
        for s, v in zip(segments, seg_counts):
            rows.append({"week_start": week_start, "date": dt, "dow": dow, "segment": s, "volume": int(v)})
    return pd.DataFrame(rows)

# Build baseline weeks (slight noise around baseline total)
dfs=[]
for ws in week_starts:
    jitter = int(rng.normal(0, 180))  # small variance
    total = max(6500, BASELINE_WEEK_TOTAL + jitter)
    dfs.append(make_week(ws, total, baseline_shares, complete=True))

# Replace last complete week with the "current" week totals + current mix
dfs.append(make_week(current_week_start, CURRENT_WEEK_TOTAL, current_shares, complete=True))

# Add partial week activity (incomplete week)
dfs.append(make_week(partial_week_start, PARTIAL_WEEK_TOTAL, current_shares, complete=False))

df = pd.concat(dfs, ignore_index=True)

# Add a simple "invalid" flag stream (tiny rate; watchlist only)
df["invalid"] = rng.binomial(1, 0.012, size=len(df))  # ~1.2% rows flagged (synthetic)
df["invalid_volume"] = np.where(df["invalid"]==1, np.minimum(df["volume"], rng.integers(1, 10, size=len(df))), 0)

# --- Helpers ---
def callout(text: str):
    if TEACH_MODE:
        display(Markdown(f"> **Interpretation:** {text}"))

@dataclass(frozen=True)
class State:
    decision: str
    comparator_validity: str
    actionability_status: str
    partial_week_ratio: float
    partial_week_risk: str
    baseline_week_total: int
    current_week_total: int
    delta_week: int
    delta_per_bus_day: float
    volume_shift_flag: bool
    history_tier: str

def compute_state() -> State:
    pct_delta = (CURRENT_WEEK_TOTAL - BASELINE_WEEK_TOTAL) / BASELINE_WEEK_TOTAL
    volume_shift_flag = abs(pct_delta) >= 0.15
    pwr = partial_week_risk_label(PARTIAL_RATIO)
    # Comparator validity in this teaching deck is CONFIRMED (complete+mature week exists)
    comparator_validity = "CONFIRMED"
    # LIMITED_CONTEXT when shift triggered + partial-week present + only 12w context
    actionability_status = "LIMITED_CONTEXT" if volume_shift_flag else "STABLE"
    decision = "HOLD queue expansion; validate persistence before actioning capacity."
    return State(
        decision=decision,
        comparator_validity=comparator_validity,
        actionability_status=actionability_status,
        partial_week_ratio=PARTIAL_RATIO,
        partial_week_risk=pwr,
        baseline_week_total=BASELINE_WEEK_TOTAL,
        current_week_total=CURRENT_WEEK_TOTAL,
        delta_week=DELTA_WEEK,
        delta_per_bus_day=DELTA_PER_BUS_DAY,
        volume_shift_flag=volume_shift_flag,
        history_tier="12w (LIMITED_CONTEXT)"
    )

state = compute_state()


# Queue Volume Shift Brief (v1.6)

*Operator decision brief for weekly queue volume shifts.*

In [ ]:

display(Markdown(f"**Data through:** {DATA_THROUGH}  \n**Generated:** {GENERATED_AT}"))


## Executive Snapshot

In [ ]:

md = []
md.append(f"**Decision:** {state.decision}")
md.append("")
md.append(f"**Impact (vs 8w baseline):** Baseline **{state.baseline_week_total:,.0f}/wk** → Current **{state.current_week_total:,.0f}/wk**  \nΔ **+{state.delta_week:,.0f}/wk** (≈ **+{state.delta_per_bus_day:,.0f}/business day**)")
md.append("")
md.append(f"**Comparator validity:** {state.comparator_validity}")
md.append(f"**Actionability status:** {state.actionability_status}")
md.append(f"**Partial-week risk:** {state.partial_week_risk} ({PARTIAL_WEEK_TOTAL:,.0f}/{state.current_week_total:,.0f} = {state.partial_week_ratio*100:.1f}%)")
md.append("")
md.append("**Why LIMITED_CONTEXT (reversible actions only):**")
md.append(f"- Volume shift trigger tripped (|%Δ| ≥ 15% vs rolling 8-week median baseline).")
md.append(f"- Partial-week activity present ({state.partial_week_ratio*100:.1f}% of anchor week) + history tier {state.history_tier}.")
md.append("")
md.append("**Next actions (reversible):**")
md.append("- Validate day-of-week pattern vs baseline band (rule out cutoff/lag effects).")
md.append("- Segment the delta (payer/LOB/driver) and triage top contributors.")
md.append("- Re-check after the next complete+mature week closes; defer capacity/staffing changes until persistence is confirmed.")
display(Markdown("\n".join(md)))


## Executive Translation (plain English)

In [ ]:

lines = [
"- **Baseline** = the typical weekly volume (rolling 8-week median).",
"- **Current** = the latest complete + mature week we trust for decisions.",
"- **Partial-week activity** = the current in-progress week (can be misleading).",
"- **Comparator validity** = we have a complete week to compare against baseline (apples-to-apples).",
"- **Actionability status (LIMITED_CONTEXT)** = we treat the signal as real enough to investigate, not real enough for irreversible changes.",
"- **Capacity index** = current ÷ baseline (how many times larger than normal).",
]
display(Markdown("\n".join(lines)))


## Micro glossary (6 terms)

In [ ]:

items = [
"- **Complete week:** 7/7 days present for the anchor week.",
"- **Mature gate:** in real ops, use settled/adjudicated window (e.g., 45+ days) before trusting volume.",
"- **Rolling 8-week median:** robust baseline resistant to one-off spikes.",
"- **Partial-week ratio:** partial-week volume ÷ anchor week volume.",
"- **Actionability status:** STABLE / INVESTIGATE / LIMITED_CONTEXT (drives reversible vs irreversible actions).",
"- **Other (mix shift):** remainder outside Top-N segments, aggregated so totals reconcile.",
]
display(Markdown("\n".join(items)))


## Data contract (required fields)

In [ ]:
rows = [
    {"Field":"week_start", "Alt name":"—", "Required?":"Yes", "Type":"date", "Used for":"weekly rollups + baselines"},
    {"Field":"date", "Alt name":"—", "Required?":"Yes", "Type":"date", "Used for":"day-of-week ramp"},
    {"Field":"dow", "Alt name":"day_of_week", "Required?":"Yes", "Type":"int (0-6)", "Used for":"DOW comparisons + bands"},
    {"Field":"segment", "Alt name":"payer / lob / driver", "Required?":"Yes", "Type":"string", "Used for":"segment deltas + mix shift"},
    {"Field":"volume", "Alt name":"count", "Required?":"Yes", "Type":"int", "Used for":"all volume KPIs"},
    {"Field":"invalid_volume", "Alt name":"—", "Required?":"No", "Type":"int", "Used for":"data quality watchlist (materiality gating)"},
    {"Field":"invalid", "Alt name":"invalid_flag", "Required?":"No", "Type":"0/1", "Used for":"invalid reason drilldown (optional)"},
]
dfc = pd.DataFrame(rows)
display(dfc)


## Impact translation (baseline vs current)

In [ ]:

# Bars: baseline vs current
fig, ax = plt.subplots(figsize=(8.5,4.5))
ax.bar(["Baseline (8w median)","Current (complete+mature)"], [state.baseline_week_total, state.current_week_total])
ax.set_ylabel("Weekly volume")
ax.set_title("Weekly volume: baseline vs current")
for i,v in enumerate([state.baseline_week_total, state.current_week_total]):
    ax.text(i, v, f"{v:,.0f}", ha="center", va="bottom")
ax.text(0.5, max(state.baseline_week_total, state.current_week_total)*0.85,
        f"Δ +{state.delta_week:,.0f}/wk  (~+{state.delta_per_bus_day:,.0f}/business day)",
        ha="center")
plt.show()
callout("Use this slide to size the magnitude. Percent change is secondary; the actionable number is Δ/day.")

# Capacity index + ops-load sizing (realistic bands)
cap_index = state.current_week_total / state.baseline_week_total
assumed = [500, 1000, 2500, 5000]
tbl=[]
for tp in assumed:
    net = state.delta_per_bus_day - tp
    tbl.append([tp, f"{net:+,.0f}"])
out = pd.DataFrame(tbl, columns=["Assumed throughput/day","Net backlog change/day"])
display(Markdown(f"Capacity index ≈ **{cap_index:.2f}×** (current ÷ baseline)."))
display(out)
display(Markdown("*Illustrative sizing input (not a staffing recommendation).*"))


## Validity Receipt

In [ ]:

line = (
    f"Data through: {DATA_THROUGH} | Anchor week: {current_week_start} "
    f"| Comparator: {state.comparator_validity} | Actionability: {state.actionability_status} "
    f"| Partial-week risk: {state.partial_week_risk} ({PARTIAL_WEEK_TOTAL:,.0f}/{state.current_week_total:,.0f} = {state.partial_week_ratio*100:.1f}%) "
    f"| History tier: {state.history_tier}"
)
display(Markdown(line))


## Week completeness ramp

In [ ]:

# Baseline DOW band (median + IQR) across baseline weeks
baseline_weeks = df[df["week_start"].isin(week_starts)]
dow_base = baseline_weeks.groupby(["week_start","dow"])["volume"].sum().reset_index()

med = dow_base.groupby("dow")["volume"].median()
q1  = dow_base.groupby("dow")["volume"].quantile(0.25)
q3  = dow_base.groupby("dow")["volume"].quantile(0.75)

# Current week DOW
cur = df[df["week_start"]==current_week_start].groupby("dow")["volume"].sum().reindex(range(7), fill_value=0)

# Cumulative
cum_med = med.cumsum()
cum_q1  = q1.cumsum()
cum_q3  = q3.cumsum()
cum_cur = cur.cumsum()

fig, ax = plt.subplots(figsize=(9,4.5))
ax.plot(cum_med.index, cum_med.values, marker="o", label="Baseline median (cum)")
ax.fill_between(cum_med.index, cum_q1.values, cum_q3.values, alpha=0.2, label="Baseline IQR band")
ax.plot(cum_cur.index, cum_cur.values, marker="o", label="Current (cum)")
ax.set_xticks(range(7))
ax.set_xticklabels(["Mon","Tue","Wed","Thu","Fri","Sat","Sun"])
ax.set_ylabel("Cumulative volume")
ax.set_title("Cumulative volume by day-of-week vs baseline band")
plt.legend()
plt.show()

# Top divergences
diff = (cur - med).sort_values(ascending=False)
top = diff.head(2)
callout(f"Largest positive divergence days: {', '.join([['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][i] for i in top.index])}. Confirm if pattern persists in next complete week.")


## Segment drivers + mix shift

In [ ]:

# Segment totals for baseline vs current
base_week_total = BASELINE_WEEK_TOTAL
cur_week_total = CURRENT_WEEK_TOTAL

base_seg = pd.Series(baseline_shares, index=segments) * base_week_total
cur_seg  = pd.Series(current_shares,  index=segments) * cur_week_total
base_seg = base_seg.round().astype(int)
cur_seg  = cur_seg.round().astype(int)
# Rounding fix
base_seg.iloc[-1] += base_week_total - base_seg.sum()
cur_seg.iloc[-1]  += cur_week_total  - cur_seg.sum()

delta = (cur_seg - base_seg).sort_values(ascending=False)
topN=3
top = delta.head(topN)
other = delta.iloc[topN:]
other_total = other.sum()

contrib = pd.DataFrame({
    "Segment": list(top.index) + ["Other"],
    "Delta (current - baseline)": list(top.values) + [int(other_total)]
})
fig, ax = plt.subplots(figsize=(9,4.5))
ax.bar(contrib["Segment"], contrib["Delta (current - baseline)"])
ax.set_ylabel("Delta volume")
ax.set_title("Top contributors to delta (Top-3 + Other)")
for i,v in enumerate(contrib["Delta (current - baseline)"]):
    ax.text(i, v, f"{v:,.0f}", ha="center", va="bottom")
plt.show()

# Mix shift (shares)
mix = pd.DataFrame({
    "Segment": segments,
    "Baseline share": baseline_shares,
    "Current share": current_shares,
})
mix["Δ share (pp)"] = (mix["Current share"] - mix["Baseline share"]) * 100
mix_sort = mix.sort_values("Current share", ascending=False)

fig, ax = plt.subplots(figsize=(9,4.5))
ax.bar(mix_sort["Segment"], mix_sort["Baseline share"]*100, label="Baseline")
ax.bar(mix_sort["Segment"], mix_sort["Current share"]*100, alpha=0.7, label="Current")
ax.set_ylabel("Share of weekly volume (%)")
ax.set_title("Mix shift: baseline vs current share")
plt.xticks(rotation=30, ha="right")
plt.legend()
plt.show()

# "Other" definition + top contributors inside Other
other_segments = list(other.index)
top_inside_other = other.abs().sort_values(ascending=False).head(2).index.tolist()
display(Markdown(
    f"Other = all remaining segments outside Top-N (ranked by |delta|), aggregated. "
    f"(N={len(other_segments)}) Top contributors inside Other: {', '.join(top_inside_other)}."
))
display(Markdown(
    f"Drivers shown: {topN} (of {len(segments)}). Other = remaining segments (N={len(other_segments)}). "
    f"Top contributors inside Other: {', '.join(top_inside_other)}."
))

callout("Use this slide to answer: ‘Where is the delta coming from?’ and ‘Did composition change?’")


## Decision + trigger rules

In [ ]:

pct_delta = (state.current_week_total - state.baseline_week_total) / state.baseline_week_total
lines = [
f"- **Decision:** {state.decision}",
f"- **Volume shift trigger:** |%Δ| ≥ 15% vs rolling 8-week median baseline (observed: {pct_delta*100:.1f}%).",
f"- **Comparator validity gate:** only compare complete+mature weeks (current is {state.comparator_validity}).",
f"- **When LIMITED_CONTEXT:** take reversible actions (validate/segment/triage) and defer capacity/staffing/process changes until persistence is confirmed.",
]
display(Markdown("\n".join(lines)))


## Decision Criteria (HOLD -> EXPAND)

In [ ]:

pct_delta = (state.current_week_total - state.baseline_week_total) / state.baseline_week_total
criteria = [
"- **Flip HOLD -> EXPAND only if ALL are true:**",
f"  1) Next complete+mature week remains ≥ +15% vs baseline (current: {pct_delta*100:.1f}%).",
"  2) Sensitivity check agrees (6w vs 8w and median vs trimmed mean do not change direction).",
"  3) DOW divergence is not a single-day artifact (≥2 days outside baseline band).",
"  4) Same top contributors account for most of the delta (Top-3 share stays dominant).",
"",
"- **If any condition fails:** keep HOLD and continue reversible actions only."
]
display(Markdown("\n".join(criteria)))


## Appendix: Data Quality Watchlist

In [ ]:

# KPI card style outputs (materiality gated)
week = df[df["week_start"]==current_week_start]
invalid_total = int(week["invalid_volume"].sum())
week_total = int(week["volume"].sum())
rate = invalid_total / max(1, week_total)

display(Markdown(f"**Invalid volume (anchor week):** {invalid_total:,.0f}  \n**Invalid rate:** {rate*100:.2f}%"))
if rate >= 0.05:
    display(Markdown("⚠️ **ALERT:** invalid rate breaches 5% threshold — investigate before trusting deltas."))
else:
    display(Markdown("No alert: invalids are below materiality threshold; monitor only."))

# Minimal triage table (top invalid segments)
tri = week.groupby("segment")[["invalid_volume","volume"]].sum().reset_index()
tri["invalid_rate_%"] = (tri["invalid_volume"] / tri["volume"] * 100).round(2)
tri = tri.sort_values(["invalid_volume"], ascending=False).head(5)
display(tri)
